# 1. Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetClassifier
import imblearn as imb

import os

### Testing Tensorflow GPU

In [ ]:
tf.test.is_built_with_cuda()

# 2. Project Variables

In [ ]:
from functions import *

In [ ]:
DATA_DIR = '../train-test-data'
NUM_FOLDS = 2
TASKS_TO_RUN = ['2aii', '2aiii']

# 3. Model Training

In [ ]:
def train_tabnet(train, test, task):
    X_train, y_train = df_to_nparray(train, task)
    X_test, y_test = df_to_nparray(test, task)

    model = TabNetClassifier()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    tp, fp, tn, fn = [0,0,0,0]
    for p,a in zip(preds, y_test):
        if (p == 'INCREASED RISK' and a == 'INCREASED RISK'): tp += 1
        elif (p == 'INCREASED RISK' and a == 'REDUCED RISK'): fp += 1
        elif (p == 'REDUCED RISK' and a == 'INCREASED RISK'): fn += 1
        else: tn += 1

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)

    return -1, accuracy, sensitivity, specificity

In [ ]:
metrics = {}

included_cols = ['CHILD_SEX','IDD_SCORE','AGE','HHID_count','HH_AGE','FOOD_EXPENSE_WEEKLY',
                 'NON-FOOD_EXPENSE_WEEKLY','HDD_SCORE','FOOD_INSECURITY','YoungBoys','YoungGirls',
                 'AverageMonthlyIncome','BEN_4PS','AREA_TYPE','FOOD_EXPENSE_WEEKLY_pc',
                 'NON-FOOD_EXPENSE_WEEKLY_pc','AverageMonthlyIncome_pc']

for task in TASKS_TO_RUN:
    metric = train_kfold(NUM_FOLDS, task, included_cols, train_nn)
    metrics[task] = metric

In [ ]:
for task in TASKS_TO_RUN:
    print(f'{task}: ACCURACY: {metrics[task]["ACCURACY"]["MEAN"]} SENSITIVITY: {metrics[task]["SENSITIVITY"]["MEAN"]} SPECIFICITY: {metrics[task]["SPECIFICITY"]["MEAN"]}')

In [ ]:
metrics['2aii']['ACCURACY']['ALL']

# Model Evaluation

### Note: This runs evaluates the models with the testing set. Run only at the end.